In [4]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd

In [5]:
df1 = pd.DataFrame([['teste1', 'teste2'],['teste3', 'teste4']])
df2 = pd.DataFrame([['A', 'B'], ['TESTE', 'teste']])
df3 = pd.DataFrame([['A', 'B'], ['TESTE', 'teste'], ['1'], ['1', '14']])
df1

,0,1
0,teste1,teste2
1,teste3,teste4


In [6]:
df2

,0,1
0,A,B
1,TESTE,teste


In [7]:
df3

,0,1
0,A,B
1,TESTE,teste
2,1,None
3,1,14


In [10]:
df = pd.concat([df1, df2], axis=0)
df


,0,1
0,teste1,teste2
1,teste3,teste4
0,A,B
1,TESTE,teste


In [18]:
df.reset_index(drop=True, inplace=True)

In [19]:
df

,0,1
0,teste1,teste2
1,teste3,teste4
2,A,B
3,TESTE,teste


In [12]:
value = 'A'

In [24]:
df[df[0] == value]

,0,1
2,A,B


In [27]:
for i in df.columns:
    
    print(df[df[i] == value])

   0  1
2  A  B
Empty DataFrame
Columns: [0, 1]
Index: []


In [2]:
def Conection (SCOPES = 'https://www.googleapis.com/auth/spreadsheets'):
    creds = None
    
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    
    return creds


In [3]:
def Read_Sheets(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME, creds)->list:
    
    
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets() ## Pegou o arquivo inteiro 
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=SAMPLE_RANGE_NAME).execute() ## aqui que passo qual planilha quero ler e qual intervalo de celulas vou ler
    values = result.get('values', []) ## esse cara que diz se quero pegar os valores ou estilização do google

    return values


In [4]:
def Update_sheets(creds,SAMPLE_SPREADSHEET_ID,values):
    ''' 
        Adicionar elementos 
        Testo isso depois  
    '''
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets() ## Pegou o arquivo inteiro 

    meses_para_datas = {
        'Jan': '01/01/2022',
        'Fev': '01/02/2022',
        'Mar': '01/03/2022',
        'Abr': '01/04/2022',
        'Mai': '01/05/2022',
        'Nov': '01/11/2022'
    }

    
    arr = [['01/01/2023', '2130'],['01/02/2023', '2000'],['01/03/2023', '1000']]
    res = []
    for mes, valor in values:
        if mes in meses_para_datas:
            res.append([meses_para_datas[mes], valor])
        else :
            res.append([mes, valor])
     

    for i in arr:
        res.append(i)
    print(res)

    result = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID, range='Tabela dinâmica 1!A1:J30', valueInputOption='USER_ENTERED', body={"values":res}).execute()
    


In [5]:
def Delete_sheets(id_sheets):
    pass

In [3]:
print(Conection(['https://www.googleapis.com/auth/spreadsheets']))

In [19]:

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1Iihqlrka23l3w6dskDxjQBKiGzLw-xYoUnzFWmmrni0'
SAMPLE_SPREADSHEET_ID_CLIENTE = '1SXhHkgpP7p9M3WTB-FtQ9rBRoxdQnDt20ZstE3QKkLc' 

SAMPLE_RANGE_NAME = 'Página1!A3:B13'
SAMPLE_RANGE_DINAMIC = 'Tabela dinâmica 1!A1:J30'


def main():
    
    creds = Conection(SCOPES) ## credenciais como retorno da conexão 
    
    try:

        # Call the Sheets API
        values = Read_Sheets(SAMPLE_SPREADSHEET_ID,SAMPLE_RANGE_NAME, creds)
        
        ## result é a requisição
        print(values)
        
        Update_sheets(creds, SAMPLE_SPREADSHEET_ID, values)

        


        # if not values:
        #     print('No data found.')
        #     return

        # print('Name, Major:')
        # for row in values:
        #     # Print columns A and E, which correspond to indices 0 and 4.
        #     print('%s, %s' % (row[0], row[4]))

    except HttpError as err:
        print(err)


In [20]:
main()

[['01/03/2023', '1000'], ['01/05/2022', '5421'], ['01/11/2022', '5642'], ['01/01/2023', '2130'], ['01/02/2023', '2000'], ['mar./1', '1000'], ['01/01/2023', '2130'], ['fev./1', '2000'], ['01/03/2023', '1000'], ['01/01/2023', '2130'], ['01/02/2023', '2000']]
[['01/03/2023', '1000'], ['01/05/2022', '5421'], ['01/11/2022', '5642'], ['01/01/2023', '2130'], ['01/02/2023', '2000'], ['mar./1', '1000'], ['01/01/2023', '2130'], ['fev./1', '2000'], ['01/03/2023', '1000'], ['01/01/2023', '2130'], ['01/02/2023', '2000'], ['01/01/2023', '2130'], ['01/02/2023', '2000'], ['01/03/2023', '1000']]
